# Download X-TRACK coastal altimetry data

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from ftplib import FTP

os.chdir("/gdata1/data/altimetry/X-TRACK 2.2/")

### Download data from Aviso+ FTP
> Altimetry data used here (v2.2, DOI 10.24400/527896/a01-2022.020) were developed, validated by the CTOH/LEGOS, France and distributed by Aviso+:
https://www.aviso.altimetry.fr/en/data/products/sea-surface-height-products/regional/x-track-sla.html

In [ ]:
# Connect and list directory contents
PASSWORD = ""
ftp = FTP("ftp-access.aviso.altimetry.fr")
ftp.login("Robbi.BishopTaylor@ga.gov.au", PASSWORD)

ftp_dirs = [
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/WAUSTRALIA/S3A/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/EAUSTRALIA/S3A/SLA",
    "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/WAUSTRALIA/S3B/SLA",
    "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/EAUSTRALIA/S3B/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/WAUSTRALIA/TP+J1+J2+J3+S6A/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/EAUSTRALIA/TP+J1+J2+J3+S6A/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/EAUSTRALIA/ERS1+ERS2+ENV+SRL/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/WAUSTRALIA/ERS1+ERS2+ENV+SRL/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/EAUSTRALIA/TPN+J1N+J2N+J3N/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/WAUSTRALIA/TPN+J1N+J2N+J3N/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/EAUSTRALIA/HY2A/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/WAUSTRALIA/HY2A/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/EAUSTRALIA/HY2B/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/WAUSTRALIA/HY2B/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/EAUSTRALIA/GFO/SLA",
    # "regional-xtrack-coastal/version_x_track_l2p_2022/v2.2/WAUSTRALIA/GFO/SLA"
]

for path in ftp_dirs:
    try:
        for nc_file in ftp.nlst(path):
            # Get file name
            fname = nc_file.split("/")[-1]

            # Download if it doesn't exist
            if not os.path.isfile(fname):
                print(f"Downloading {fname}")
                with open(fname, "wb") as f:
                    ftp.retrbinary(f"RETR {nc_file}", f.write)
            else:
                print(f"Skipping {fname}")
    except Exception as e:
        print(f"Skipping {path}: {e}")

### Experiments

In [ ]:
import xarray as xr

ds1 = xr.open_mfdataset("ctoh.sla.ref.S3A.waustralia.247.nc").sel(points_numbers=1).time.compute()
time1 = ds1.where(ds1.dt.year.isin([2017, 2018, 2019]), drop=True)

ds2 = xr.open_mfdataset("ctoh.sla.ref.S3B.waustralia.247.nc").sel(points_numbers=1).time.compute()
time2 = ds2.where(ds2.dt.year.isin([2017, 2018, 2019]), drop=True)

In [ ]:
import datetime

import matplotlib.pyplot as plt


# Convert cftime.DatetimeJulian → datetime.datetime
def convert_to_datetime(cftime_array):
    return [datetime.datetime.strptime(t.strftime("%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S") for t in cftime_array]


dates1 = convert_to_datetime(time1.values)
dates2 = convert_to_datetime(time2.values)

# Plotting
plt.figure(figsize=(10, 2))
plt.scatter(dates1, [1] * len(dates1), color="red", label="Time Series 1")
plt.scatter(dates2, [2] * len(dates2), color="blue", label="Time Series 2")

plt.xlabel("Date")
plt.ylabel("Value")
plt.title("Two Time Series from cftime.DatetimeJulian")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()